In [1]:
import os

In [2]:
%pwd

'c:\\Users\\91902\\OneDrive\\Desktop\\ml-end-to-end\\end-end-ml\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\91902\\OneDrive\\Desktop\\ml-end-to-end\\end-end-ml'

In [17]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngesionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [18]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [19]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngesionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngesionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [20]:
import os
import urllib.request as request
import zipfile
from mlProject.Logging import logger
from mlProject.utils.common import get_size

In [21]:
class DataIngestion:
    def __init__(self, config: DataIngesionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [22]:

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-01-18 12:39:44,275:INFO:common:YAML file: config\config.yaml loaded successfully]
[2025-01-18 12:39:44,277:INFO:common:YAML file: params.yaml loaded successfully]
[2025-01-18 12:39:44,282:INFO:common:YAML file: schema.yaml loaded successfully]
[2025-01-18 12:39:44,284:INFO:common:Created directory at: artifacts]
[2025-01-18 12:39:44,285:INFO:common:Created directory at: artifacts/data_ingestion]
[2025-01-18 12:39:45,719:INFO:957573136:artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 21984
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "85a38be3d4c70e3b6dc97fc47873623da88c9b850e1526aeb57ff236bdd2815c"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4C48:151AA:192BA0:24CE7D:678B53B9
Accept-Ranges: bytes
Date: Sat, 18 Jan 2025 07:09:46 G